# Исследование надёжности заёмщиков



Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.info()
display(data.columns)
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Информация о таблице показывает нам, что есть пропущенные данные в двух столбцах: 'days_employed' и 'total_income'. Количество строк, в которых данные по этим столбцам отсутствуют, равно. Возможно, эти пропуски встречаются попарно и зависят от других данных в датафрейме.
Данные столбцов 'days_employed' и 'total_income' представлены в float64, их необходимо преобразовать в int64.
Часть данных в столбе 'days_employed' имеют отрицательное значение, что является явной ошибкой. Их можно взять по модулю, так как в будущем они могут нам помочь в поиске строк-дубликатов в датасете. Так же при возникновинии подобных ситуаций следует обращаться к источнику данных для того, чтобы эти ошибки не повторялись в будущем. В 'education' указание образования встречается с разным регистром и их необходимо привести к общему виду - к нижнему регистру.

## Предобработка данных

### Обработка пропусков

In [4]:
data['days_employed'] = data['days_employed'].fillna(0)
display(data['income_type'].unique())

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [5]:
income_type = data['income_type'].drop_duplicates().to_list()
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод

В данном наборе данных пропущены значения только в столбцах 'days_employed' и 'total_income', причем пропущены они попарно. Вполне вероятно, что случайный пропуск(или пропуск при переносе из одного источника в другой) в ячейке первого столбца повлёк за собой пропуск данных и в ячейке второго столбца.
Для решения поставленных задач данные столбца 'days_employed' не играют роли, поэтому пустые значения можно заменить на '0'.
А на данные столбца 'total_income' мы будем полагаться при ответе на один из поставленных перед нами вопросов, поэтому пропущенные данные можно заполнить медиальным значением категории 'income_type'.
Заполняем пропущенные значения при помощи метода fillna()

### Замена типа данных

In [6]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

Указанные данные в столбцах 'days_employed' и 'total_income' не требуют такой точности, которую дает тип float64, поэтому их можно преобразовать в int64 методом astype().

### Обработка дубликатов

In [7]:
# функция приводящая отрицательные числовые значения к модулю, а текстовые - к нижнему регистру
def one(df):
    for i in df.columns:
        try:
            if df[i].min() < 0:
                df[i] = df[i].abs()
        except:
            df[i] = df[i].str.lower()

In [8]:
one(data)
data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

Перед обработкой дубликатов необходимо избавиться от заведомо ошибочных отрицательных значений в числовых данных датафрейма и от использования различного регистра написания букв в текстовых данных. Всё это делается при помощи одной функции, использующей внутри себя "try-except", где в блоке "try" обрабатываются числовые значения, а в "except" - текстовые.
После этого были найдены и удалены дубликаты, был использован метод drop_duplicates(). Данный метод удаляет только дублирующие строки датафрейма, оставляя их индексы. Для сброса индексов используем метод reset_index().

### Лемматизация

In [9]:
m = Mystem()
# функция на вход получает текст, лемматизирует его, склеивает все лемматизированные слова в одну строку
# и возвращает эту строку
def lemmfunc(txt): 
    lemmas = ''.join(m.lemmatize(txt)).replace("\n","")
    return lemmas

In [10]:
data['lemmas'] = data['purpose'].apply(lemmfunc)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,покупка жилье
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,операция с жилье
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья


#### Вывод

Для создания категорий целей кредита воспользуемся данными, указанными в столбце 'purpose'. Однако перед этим их нужно лемматизации, чтобы потом их можно было посчитать. При лемматизации используем библиотеку PyMystem, а точнее её метод lemmatize(). 

### Категоризация данных

In [11]:
purpose = ' '.join(data['lemmas'].tolist()).split()
display(Counter(purpose))
        
display(data['total_income'].describe(percentiles=[0.33,0.66]))

Counter({'покупка': 5897,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
33%      1.185140e+05
50%      1.425940e+05
66%      1.723570e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [12]:
#функция для создания категорий целей кредитов
def purpose_category(txt): 
    for word in txt.split(' '):
        if word == 'недвижимость':
            return 'недвижимость'
        if word == 'жилье':
            return 'жилье'
        if word == 'автомобиль':
            return 'автомобиль'
        if word == 'образование':
            return 'образование'
        if word == 'свадьба':
            return 'свадьба'
        if word == 'ремонт':
            return 'ремонт'

In [13]:
#функция  для создания категорий уровня заработка
def total_income_category(x): 
    if x < 122163:
        return 'Низкий доход'
    if 122163 <= x <= 169339:
        return 'Средний доход'
    return 'Высокий доход'

In [14]:
data['purpose_category'] = data['lemmas'].apply(purpose_category) 
data['total_income_category'] = data['total_income'].apply(total_income_category)

#### Вывод

Для ответов на вопросы зависимости сроков погашения кредитов от целей кредита и уровня дохода необходимо создать категории по целям кредита и уровню дохода.
Для первой категории сначала подсчитываем частоту упоминания слов столбца 'lemmas' и анализируем результат. У нас получается 6 категорий: недвижимость, жилье, автомобиль, образование, свадьба и ремонт. Используя функцию purpose_category() в каждой строке на основании данных столбца 'lemmas' устанавливаем категорию в столбец 'purpose_category'.
Категории уровня заработка строим на основании диапазонов величин доходов с помощью функции total_income_category().
Затем используя метод apply() и вышеуказанные функции создаем и заполняем столбцы категорий.

## Ответим на следующие вопросы:

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
# функция для создания сводной таблицы, показывающей зависимость возврата кредита в срок от определенного признака
def share_of_timely_payment(df, col):
    data_pivot = df.pivot_table(index = col, columns = 'debt', values = 'days_employed', aggfunc = 'count')
    data_pivot['share_of_timely_payment'] = data_pivot[0]/(data_pivot[0]+data_pivot[1])
    return data_pivot.sort_values(by = 'share_of_timely_payment', ascending = False).style.format({'share_of_timely_payment': '{:.2%}'})

In [16]:
share_of_timely_payment(data, 'children')

debt,0,1,share_of_timely_payment
children,,,
0,13028.000000,1063.000000,92.46%
3,303.000000,27.000000,91.82%
1,4410.000000,445.000000,90.83%
2,1858.000000,194.000000,90.55%
4,37.000000,4.000000,90.24%
20,68.000000,8.000000,89.47%
5,9.000000,nan,nan%


#### Вывод

На основании полученных данных можно сделать вывод, что шанс возврата кредита в срок больше, если у заёмщика отсутствуют дети и этот шанс уменьшается по мере роста количества детей.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
share_of_timely_payment(data, 'family_status')

debt,0,1,share_of_timely_payment
family_status,,,
вдовец / вдова,896,63,93.43%
в разводе,1110,85,92.89%
женат / замужем,11408,931,92.45%
гражданский брак,3763,388,90.65%
не женат / не замужем,2536,274,90.25%


In [18]:
share_of_timely_payment(data, 'family_status')

debt,0,1,share_of_timely_payment
family_status,,,
вдовец / вдова,896,63,93.43%
в разводе,1110,85,92.89%
женат / замужем,11408,931,92.45%
гражданский брак,3763,388,90.65%
не женат / не замужем,2536,274,90.25%


#### Вывод

На основании собранных данных можно сделать вывод, что шанс возврата кредита в срок выше, если заемщик находится или некогда состоял в браке.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
share_of_timely_payment(data, 'total_income_category')

debt,0,1,share_of_timely_payment
total_income_category,,,
Высокий доход,7212,585,92.50%
Низкий доход,6853,613,91.79%
Средний доход,5648,543,91.23%


#### Вывод

На основании полученных нельзя сделать однозначный вывод относительно зависимости между уровнем дохода и возвратом кредита в срок.

### - Как разные цели кредита влияют на его возврат в срок?

In [20]:
share_of_timely_payment(data, 'purpose_category')

debt,0,1,share_of_timely_payment
purpose_category,,,
ремонт,572,35,94.23%
жилье,3580,273,92.91%
недвижимость,5877,474,92.54%
свадьба,2138,186,92.00%
образование,3643,370,90.78%
автомобиль,3903,403,90.64%


#### Вывод

На основании полученных данных можно сделать вывод, что наиболее вероятен возврат кредита в срок, если целью кредита была покупка или ремонт недвижимости. Чуть отстает по вероятности возврата в срок кредит с целью "свадьба". И наименьшими шанcами возврата в срок обладают кредиты на образование или автомобиль.

## Общий вывод

В исходном датасете присутствовали ошибки и пропуски в данных. Были обработаны только те из них, которые имели значение при выполнении поставленной задачи. Пропуски в стаже были заполнены нулями, так как данные по стажу не играют роли в нашем случае, но пропуски помешали бы в дальнейшей обработке датасета. Пропуски в доходах были заполнены медианными значениями категорий из столбца 'income_type'. 
Для уменьшения размера таблицы данные, не требующие большой точности, были переведены из формата с плавающей запятой в целочисленный вид.
Были убраны дубликаты из нашего набора данных, перед этим все числа таблицы были взяты по модулю, а текстовые значения переведены в нижний регистр.

Что же касается поставленных задач, то можно говорить, что шанс возврата кредита в срок зависит от:
- **наличия детей** - чем их больше, там меньше шанс своевременного погашения;
- **семейного положения** - шанс больше, если человек находится или когда-либо вступал в брак;
- **целей кредита** - наибольшим процентом погашения в срок обладает кредит, взятый на покупку или ремонт недвижимости, чуть отстает кредит, взятый на свадьбу.

Обобщив вышеописанное можно описать идеального клиента кредитного отдела банка - это человек, не имеющий детей, когда-либо состоявший в браке и берущий кредит на покупку или ремонт недвижимости.
Наглядно это можно увидеть в шаге 3, где в каждой таблице привлекательность определенных групп пользователей кредитных продуктов определяется процентом возвратов кредитов в срок.

Представителям заказчика, ответственным за выгрузку данных для анализа рекомендуется проверить средства выгрузки данных и привести их к определенному формату: числовые значения не должны быть отрицательными, а текстовые должны быть в нижнем регистре. Так же необходимо обратить внимание на попарные пропуски в столбцах 'days_employed' и 'total_income', вероятно при выгрузке имеется какая-то закономерность, вследствии которой появляются эти пропуски.